In [10]:
import sys
sys.path.append('../../GoogleNewsClassfication/')

In [11]:
import pandas as pd
import plotly.express as px
#from Scripts.Preprocessing import pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB

In [12]:
import numpy as np 

In [13]:
df = pd.read_csv('../data/clean_data.csv')

In [14]:
df.head()

,Title,Category
0,chainlink link falter hedera hbar wobbl yet vc...,Business
1,fund punish nvidia share stun 230 ralli,Business
2,crude oil price stall hedg fund sell kemp,Business
3,grayscal bitcoin win still half battl,Business
4,im home shop editor cantmiss labor day deal im...,Business


In [15]:
X = df['Title']
y = df['Category']

While we know TD-IDF will be better, let's try the CountVectorizer first
* 1）Count Vectors + MultinomialNB()

In [16]:
vectorizer = CountVectorizer()
train_test = vectorizer.fit_transform(X)

In [17]:
bag_of_words_df = pd.DataFrame(train_test.toarray())
bag_of_words_df

,0,1,2,3,4,5,6,7,8,9,...,24536,24537,24538,24539,24540,24541,24542,24543,24544,24545
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51297,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51298,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
label_encoder = LabelEncoder()
df['Category_enc'] = label_encoder.fit_transform(y)

In [19]:
X =bag_of_words_df 
y = df['Category_enc']

test_size=0.2
random_state=42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

In [20]:
clf1 = MultinomialNB()
clf1.fit(X_train,y_train)

MultinomialNB()

In [21]:
val_pred1 = clf1.predict(X_test)
print(f1_score(y_test, val_pred1, average='macro'))

0.867224129687402


In [22]:
y.unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

## ***Results visualization***

In [25]:
class_labels = label_encoder.inverse_transform(range(8))
# styling the confusion matrix
confusion_matrix_kwargs = dict(
    text_auto=True,
    title="Confusion Matrix", width=1000, height=800,
    labels=dict(x="Predicted", y="True Label"),
    x=class_labels,
    y=class_labels,
    color_continuous_scale='Blues'
)

def report(y_true, y_pred, class_labels):
    print(classification_report(y_true, y_pred, target_names=class_labels))
    # print a classification report of the predictions # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn-metrics-classification-report
    # create a confusion matrix and pass it to imshow to visualize it # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix
    # (the confusion_matrix_kwargs are here for styling only)
    conf_matrix = confusion_matrix(y_true, y_pred)
    fig = px.imshow(
        conf_matrix,
        **confusion_matrix_kwargs
        )
    fig.show()

In [26]:
class_labels = label_encoder.inverse_transform(range(8))
# call the report function to visualize the classification results, use the above class_labels
print(f"f1_score:{f1_score(y_test, val_pred1, average='macro')}")
report(y_test, val_pred1, class_labels)


f1_score:0.867224129687402
               precision    recall  f1-score   support

     Business       0.87      0.82      0.84      1380
Entertainment       0.91      0.91      0.91      1384
    Headlines       0.76      0.67      0.71      1420
       Health       0.89      0.94      0.91       943
      Science       0.89      0.92      0.91       902
       Sports       0.95      0.96      0.95      1592
   Technology       0.93      0.94      0.94      1351
    Worldwide       0.73      0.81      0.77      1289

     accuracy                           0.87     10261
    macro avg       0.87      0.87      0.87     10261
 weighted avg       0.87      0.87      0.87     10261



We can see that the result is good,but there are some problems like worldwide and headlines, the model can not idenfity well.

### Let's try another Way to build a now model with new feature engineering(TF-IDF)

In [27]:
class TextClassifier:
    
    def __init__(self, model, vectorizer=None):
        self.model = model 
        self.vectorizer = vectorizer 
        self.X_train_tf = None
        self.X_test_tf = None
        
    def train(self, X_train, y_train):
        self.X_train_tf = self.vectorizer.fit_transform(X_train)
        self.model.fit(self.X_train_tf, y_train)
        
    def evaluate(self, X_test, y_test):
        self.X_test_tf = self.vectorizer.transform(X_test)
        
        y_pred = self.model.predict(self.X_test_tf)
        acc = accuracy_score(y_test, y_pred)
        print(f"f1_score:{f1_score(y_test, y_pred, average='macro')}")
        report = classification_report(y_test, y_pred)
        
        print(f"Accuracy: {acc}")
        print(report)
        confusion_matrix_kwargs = dict(
            text_auto=True,
            title="Confusion Matrix", width=1000, height=800,
            labels=dict(x="Predicted", y="True Label"),
            x=class_labels,
            y=class_labels,
            color_continuous_scale='Blues'
        )
        confusion_matrix_data = confusion_matrix(y_test, y_pred)
        fig = px.imshow(
            confusion_matrix_data,
            **confusion_matrix_kwargs
            )
        fig.show()
        
    
    def predict(self, text):
        text_tf = self.vectorizer.transform([text])
        return self.model.predict(text_tf)[0]

In [28]:
X = df['Title']
y = df['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = TextClassifier(model=MultinomialNB(), vectorizer=TfidfVectorizer(ngram_range=(1, 3), max_features=30000))
model.train(X_train, y_train)
model.evaluate(X_test, y_test)

f1_score:0.8683331340851003
Accuracy: 0.8675567683461651
               precision    recall  f1-score   support

     Business       0.85      0.84      0.85      1380
Entertainment       0.90      0.92      0.91      1384
    Headlines       0.76      0.67      0.71      1420
       Health       0.90      0.92      0.91       943
      Science       0.94      0.87      0.90       902
       Sports       0.93      0.96      0.95      1592
   Technology       0.93      0.95      0.94      1351
    Worldwide       0.73      0.81      0.77      1289

     accuracy                           0.87     10261
    macro avg       0.87      0.87      0.87     10261
 weighted avg       0.87      0.87      0.87     10261



### Now this is a simple conclusion that TF-IDF is better than CountVectorizer,so i will change the parameter like(ngram_range, max_features)

TF-IDF + RidgeClassifier

In [29]:
X = df['Title']
y = df['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = TextClassifier(model=RidgeClassifier(), vectorizer=TfidfVectorizer(ngram_range=(1, 3), max_features=30000))
model.train(X_train, y_train)
model.evaluate(X_test, y_test)

f1_score:0.882593482132392
Accuracy: 0.8813955754799727
               precision    recall  f1-score   support

     Business       0.88      0.87      0.88      1380
Entertainment       0.93      0.93      0.93      1384
    Headlines       0.77      0.67      0.71      1420
       Health       0.91      0.94      0.93       943
      Science       0.92      0.94      0.93       902
       Sports       0.96      0.97      0.97      1592
   Technology       0.96      0.95      0.96      1351
    Worldwide       0.73      0.80      0.76      1289

     accuracy                           0.88     10261
    macro avg       0.88      0.88      0.88     10261
 weighted avg       0.88      0.88      0.88     10261



For now, It's the best

### Looks like these models are good but these models above have all the same problem<br>

### They can not understand well the Worldwide and Headlines.<br>

### I have to think a way to fix this problem.

:):):):):)

### **NOW, We use the class weights to fix the problem**

To balance classes in an imbalanced dataset, we can consider using oversampling or subsampling methods to ensure that the model performs better on all classes. we can  apply class weights, which is particularly useful for imbalanced datasets.

In [30]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights for each class
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Create a dictionary associating class labels with their corresponding weights
class_weight_dict = {class_label: weight for class_label, weight in zip(np.unique(y_train), class_weights)}

# Create a classifier and set the class weight
model = TextClassifier(model=RidgeClassifier(class_weight=class_weight_dict), vectorizer=TfidfVectorizer(ngram_range=(1, 3), max_features=30000))
model.train(X_train, y_train)
model.evaluate(X_test, y_test)

f1_score:0.8836903337031135
Accuracy: 0.882662508527434
               precision    recall  f1-score   support

     Business       0.88      0.87      0.87      1380
Entertainment       0.93      0.93      0.93      1384
    Headlines       0.78      0.66      0.71      1420
       Health       0.90      0.95      0.93       943
      Science       0.92      0.95      0.93       902
       Sports       0.96      0.97      0.97      1592
   Technology       0.96      0.95      0.95      1351
    Worldwide       0.73      0.81      0.77      1289

     accuracy                           0.88     10261
    macro avg       0.88      0.89      0.88     10261
 weighted avg       0.88      0.88      0.88     10261



It's seems that il's better but not enough. Always there are two categories are fixed.